## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- Lemmatisation
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données


*Suivi des modifications apportées*  
2022-07-04
- Étiquetage morpho-syntaxique et Lemmatisation plus tôt dans le traitement pour améliorer la performance et la rapidité des deux outils.


2022-07-05
- Échantillonnage aléatoire du corpus (pour ne pas traiter la totalité des documents)
- (Log)-likelihood ratio (à suivre)

### **Lire le corpus** 

In [1]:
import shutil, re
from os import listdir, chdir, path
from pathlib import Path

acteurs = ['asso_ordres', 'chsld', 'chu_iu', 'cisss_ciusss', 'cliniques_medicales', 'csbe', 'gmf', 'inesss', 'inspq', 'msss', 'ophq', 'quebec_sante', 'ramq', 'sante_mtl', 'urgence_sante']
acteur = 'chum'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    base_path = '../03-corpus/2-sous-corpus/'
    file_path = path.join(base_path, acteur, tag)

else: 
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur) + '.csv'

In [2]:
from pandas import *
with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path)
        text = data['text'].tolist()
        corpus = [(re.sub('\d', '', t.strip('\n').lower().replace('’', '\''))) for t in text]

In [3]:
nb_docs = len(corpus)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 2299 documents.


### **Extraire un échantillon aléatoire**  
On a vu (`0-Description_corpus.ipynb`) qu'à partir d'un certain nombre de documents traités, le nombre de nouvelles formes extraites plafonne et qu'il est donc inutilement trop computationnellement coûteux de traiter la totalité de notre corpus.  

On va donc plutôt retenir seulement un échantillon de documents (environ 50% de la taille totale) extraits aléatoirement.

In [4]:
import random

n = round(len(corpus)/1) # On travaille sur environ 100% des données
corpus = random.sample(corpus, n)

print("On va travailler avec un échantillon de {} documents".format(len(corpus)))

On va travailler avec un échantillon de 2299 documents


### **Segmentation** (phrases)

**NLTK**\
https://www.nltk.org/ 

In [5]:
import nltk
#nltk.download(['popular'])

In [6]:
from nltk import sent_tokenize

sents = [[s.strip('.') for s in sent_tokenize(doc)] for doc in corpus]

In [7]:
nb_sents = len(nltk.flatten(sents))

print("Notre corpus contient {} phrases.".format(nb_sents))

Notre corpus contient 22346 phrases.


In [8]:
punct = '!#$%&()*+,-/:;<=>?@[\]^_{|}~©'

for t in punct:
    sents = [[sent.replace(t, ' ').replace("  ", " ") for sent in doc] for doc in sents]

### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [9]:
file_path = '../04-filtrage/mwe_stopwords.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mwe_sw = [t.lower().strip('\n') for t in f.readlines()]

In [10]:
for mwe in mwe_sw:
    sents = [[sent.replace(mwe, ' ').replace('  ', ' ') for sent in doc] for doc in sents]

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [11]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

tokens = [[tokenizer_re.tokenize(s) for s in doc] for doc in sents]
len_corpus = len(nltk.flatten(tokens))

print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

Avec le RegExpTokenizer, notre corpus contient 781914 tokens.


In [12]:
sents = [[" ".join(sent).replace("' ", "'") for sent in doc] for doc in tokens]

In [13]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [14]:
file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

with open(file_path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [15]:
tagged = [[[[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(sent)] for sent in doc] for doc in sents]

#if len(t.split('\t')) >1

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [16]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [17]:
lemmatizer = FrenchLefffLemmatizer()

In [18]:
lemmas = []
for doc in tagged:
    doc_l = []
    for phrase in doc:
        phrase_l = []
        for term in phrase:
            term_l = []
            if lemmatizer.lemmatize(term[0], term[1]) == []:
                term_l = [lemmatizer.lemmatize(term[0]), term[1]]
            
            elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
                term_l  = [lemmatizer.lemmatize(term[0], term[1]), term[1]]

            else:
                term_l = list(lemmatizer.lemmatize(term[0], term[1])[0])
    
            phrase_l.append(term_l)
        doc_l.append(phrase_l)
    lemmas.append(doc_l)
    

### **Filtrage** (antidictionnaire)  

In [20]:
# Importer l'antidictionnaire pour filtrer les données

# Stopwords lemmatisés
file_path = '../04-filtrage/stopwords_lemmatized.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

### **Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook

In [21]:
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.probability import FreqDist

In [36]:
ngrammes_lemmatized = [[list(everygrams(sent, min_len=2, max_len=5)) for sent in doc] for doc in lemmas]
ngrammes = [[list(everygrams(sent, min_len=2, max_len=5)) for sent in doc] for doc in tagged]

In [37]:
def filtrer_phrases(liste):
        return [ng for ng in liste if freq[ng[0]] > 30 and \
                not ng[0].split()[0] in stopwords and len(ng[0].split()[0]) > 2 \
                and not ng[0].split()[-1] in stopwords and len(ng[0].split()[-1]) > 2 \
                and not (ng[0].split()[0] == ng[0].split()[-1])]

### **Filtrage (N-grammes)**

On retire les n-grammes qui apparaissent moins de 30 fois dans tout le corpus ou qui débutent ou terminent par :
- un stopword
- un mot de 1 lettre ou moins

Pour le reste du traitement, on arrête de considérer les frontières entre les phrases et entre les documents (nos ngrammes les respectent donc on n'en a plus besoin)

In [38]:
# Calculer la distribution de fréquence de chaque ngramme dans tout le corpus
# Pour ça, on va 'applatir' la liste des ngrammes pour ne plus tenir compte des frontières entre les phrases et entre les documents
def ng_flat(ngramme):
    liste    = [] 
    for doc in ngramme:
        for sent in doc:
            for ngram in sent:
                liste.append(ngram)
    return liste

ngrammes_lemmatized = ng_flat(ngrammes_lemmatized)
ngrammes = ng_flat(ngrammes)

In [39]:
def extract_patterns(ngrammes):
    patterns = []

    for ng in ngrammes:
        phrase = []
        pattern = []
        for t in ng:
            phrase.append(t[0]) # token
            pattern.append(t[1]) # POS tag

        patterns.append([phrase, pattern])
        
    return patterns

phrases = extract_patterns(ngrammes)
phrases_lemmatized = extract_patterns(ngrammes_lemmatized)

In [41]:
freq = FreqDist([" ".join(phrase[0]) for phrase in phrases])
freq_lemmatized = FreqDist([" ".join(phrase[0]) for phrase in phrases_lemmatized])

In [43]:
def filtrer_phrases(liste):
        return [ng for ng in liste if freq[" ".join(ng[0])] > 30 and \
                not ng[0][0] in stopwords and len(ng[0][0]) > 2 \
                and not ng[0][-1] in stopwords and len(ng[0][-1]) > 2 \
                and not (ng[0][0] == ng[0][-1])]

phrases = filtrer_phrases(phrases)
phrases_lemmatized = filtrer_phrases(phrases_lemmatized)

In [45]:
freq = FreqDist([" ".join(phrase[0]) for phrase in phrases])
freq_lemmatized = FreqDist([" ".join(phrase[0]) for phrase in phrases_lemmatized])

In [46]:
vocabulaire = freq.keys()
vocabulaire_lemmatized = freq_lemmatized.keys()

def tabCSV(freqd, titre):
    base_path = '../04-filtrage/output/'
    tab = DataFrame(freqd.items(), columns= ["Expression", "Fréquence"])
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    file_path = path.join(base_path, acteur, acteur)
    if sous_corpus:
       file_path = path.join(base_path, acteur, tag, tag)
    

    Path(file_path).mkdir(parents=True, exist_ok=True)

    tab.to_csv(file_path + titre)

In [47]:
tabCSV(freq,'_n-grams.csv')
tabCSV(freq_lemmatized, '_n-grams-lemmatized.csv')

### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [48]:
file_path = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:50] # Pour prendre seulement les 50 structures syntaxiques les plus fréquentes dans les MeSH

FileNotFoundError: [Errno 2] No such file or directory: '../04-filtrage/MeSH/mesh_patterns-fr.csv'

In [50]:
cwd()

NameError: name 'cwd' is not defined

In [ ]:
terms = [t for t in phrases if t[1] in patterns]
terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

Voir combien ça a filtré en % 

In [ ]:
import pandas as pd

def extract_terms(liste, titre):
    file_path = '../04-filtrage/'
    tab = pd.DataFrame(liste, columns= ["Expression", "Structure syntaxique"])
    tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    if sous_corpus:
        file_path = path.join(file_path, acteur, tag, tag)

    else :
        file_path = path.join(file_path, acteur, acteur)

                    
    tab.to_csv(file_path + titre)

extract_terms(terms, '_terms.csv')
extract_terms(terms_lemmatized, '_terms-lemmatized.csv')

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]) for ng in ngrammes]

In [ ]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])


extrant = extrant[extrant['Fréquence'] > 30] 

file_path = '../04-filtrage/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)


extrant.to_csv(file_path + '_KWIC' +'.csv')

### **Filtrage (fréquence)**
Pour la suite du traitement, on ne retient que les N expressions (lemmatisées ou non) les plus fréquentes dans le corpus

In [ ]:
freq_terms = FreqDist([t[0] for t in terms])
exp_freq = freq_terms.most_common(10000) # On garde un maximum de 10000 termes

freq_terms_lemmatized = FreqDist([t[0] for t in terms_lemmatized])
exp_freq_lemmatized = freq_terms_lemmatized.most_common(10000) # On garde un maximum de 10000 termes

In [ ]:
base_path = '../05-transformation/'
file_path = path.join(base_path, acteur)

if sous_corpus:
    file_path = path.join(file_path, tag, tag)

else:
    file_path = path.join(file_path, acteur)

                    
Path(file_path).mkdir(parents=True, exist_ok=True)
tab = pd.DataFrame([t[0] for t in exp_freq], columns=["Terme"])
tab.to_csv(file_path + '_vocab.csv')

In [ ]:
tab = pd.DataFrame([t[0] for t in exp_freq_lemmatized], columns=["Terme"])
tab.to_csv(file_path + '_vocab-lemmatized.csv')

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize([t[0] for t in exp_freq])

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
file_path = '../04-filtrage/'
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag)

else :
    file_path = path.join(file_path, acteur, acteur)

df = DataFrame(termes_mesh)
df.to_csv(file_path + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/SNOMED/SNOMED_fr.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in exp_freq])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
file_path = '../04-filtrage/' 
if sous_corpus:
    file_path = path.join(file_path, acteur, tag, tag) 

else :
    file_path = path.join(file_path, acteur, acteur)


df = DataFrame(termes_sm)

df.to_csv(file_path + '_SNOMED.csv')